# 1. Pre-processing

#### 1.1. Text Pre-processing

The dataset is already lowercased and lacks punctuation. We will tokenize the text and remove stopwords, as well as apply lemmatization to the tokens.

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

import nltk

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

data = pd.read_json('./data/data.jsonl', lines=True)
def pre_process_data(dataset):

    #tokenize
    dataset['text'] = dataset['text'].apply(nltk.word_tokenize)

    #remove stop words
    stop_words = set(stopwords.words('english'))
    dataset['text'] = dataset['text'].apply(lambda x: [word for word in x if word not in stop_words])

    #lemmatize
    lemmatizer = WordNetLemmatizer()
    dataset['text'] = dataset['text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    
    return dataset

data = pre_process_data(data)
data.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fabio\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fabio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fabio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,text,label
0,"[feel, awful, job, get, position, succeed, hap...",0
1,"[im, alone, feel, awful]",0
2,"[ive, probably, mentioned, really, feel, proud...",1
3,"[feeling, little, low, day, back]",0
4,"[beleive, much, sensitive, people, feeling, te...",2


In [8]:
test_data = pd.read_json('./data/test.jsonl', lines=True)
train_data = pd.read_json('./data/train.jsonl', lines=True)
validation_data = pd.read_json('./data/validation.jsonl', lines=True)

test_data = pre_process_data(test_data)
train_data = pre_process_data(train_data)
validation_data = pre_process_data(validation_data)


# 2. Vectorization (secção possívelmente temporária, mas queria experimentar as cenas de tf_idf depois do pré-processamento)



In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_features=40000)
X = vectorizer.fit_transform(data['text'].apply(lambda x: ' '.join(x)))
# good idea to use two-grams??
print(X.shape)

(416809, 40000)


In [21]:
print(vectorizer.get_feature_names_out())

['aa' 'aaron' 'ab' ... 'zoom class' 'zoom grey' 'zumba']


In [67]:
X_train = vectorizer.transform(train_data['text'].apply(lambda x: ' '.join(x)))
X_val = vectorizer.transform(validation_data['text'].apply(lambda x: ' '.join(x)))
X_test = vectorizer.transform(test_data['text'].apply(lambda x: ' '.join(x)))

In [68]:
y_train = train_data['label']
y_val = validation_data['label']
y_test = test_data['label']

### 2.1. SMOTE

In [63]:
#check class distribution in training data
print(y_train.value_counts())


label
1    5362
0    4666
3    2159
4    1937
2    1304
5     572
Name: count, dtype: int64


Since there is a substantial class imbalance (a full order of magnitude between most common and least common class), we will use SMOTE to balance the classes.

In [69]:
smote = SMOTE(random_state=42, sampling_strategy='auto', k_neighbors=10)
X_train, y_train = smote.fit_resample(X_train, y_train)

print(y_train.value_counts())

label
0    5362
3    5362
2    5362
5    5362
4    5362
1    5362
Name: count, dtype: int64



# 3. Model Training

#### 3.1. Model Selection
Aqui também só estava a querer espetar modelos para começar a ver o que dá que ainda não sei que features vão ser usadas:
tf-idf, word embeddings, ???, features mais feitas à mão?


Isto pelos vistos é uma cena, que não implementei (ainda..)

"The validation set uses a subset of the training data to provide an unbiased evaluation of a model. The validation data set contrasts with training and test sets in that it is an intermediate phase used for choosing the best model and optimizing it. It is in this phase that hyperparameter tuning occurs."

Wikipedia:
The basic process of using a validation data set for model selection (as part of training data set, validation data set, and test data set) is:

Since our goal is to find the network having the best performance on new data, the simplest approach to the comparison of different networks is to evaluate the error function using data which is independent of that used for training. Various networks are trained by minimization of an appropriate error function defined with respect to a training data set. The performance of the networks is then compared by evaluating the error function using an independent validation set, and the network having the smallest error with respect to the validation set is selected. This approach is called the hold out method. Since this procedure can itself lead to some overfitting to the validation set, the performance of the selected network should be confirmed by measuring its performance on a third independent set of data called a test set.

An application of this process is in early stopping, where the candidate models are successive iterations of the same network, and training stops when the error on the validation set grows, choosing the previous model (the one with minimum error).

##### 3.1.1. Logistic Regression

In [70]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

logreg_classifier = LogisticRegression(max_iter=1000)
logreg_classifier.fit(X_train, y_train)
y_pred = logreg_classifier.predict(X_val)
print(accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred)) 

0.8995
              precision    recall  f1-score   support

           0       0.94      0.93      0.93       550
           1       0.92      0.92      0.92       704
           2       0.80      0.88      0.84       178
           3       0.92      0.89      0.91       275
           4       0.86      0.82      0.84       212
           5       0.74      0.81      0.78        81

    accuracy                           0.90      2000
   macro avg       0.86      0.88      0.87      2000
weighted avg       0.90      0.90      0.90      2000

[[511  10   4   9  13   3]
 [  9 646  35   4   3   7]
 [  2  16 157   2   1   0]
 [ 14   9   1 246   5   0]
 [  7  13   0   6 173  13]
 [  2   6   0   1   6  66]]


##### 3.1.2. Multinomial Naive Bayes

In [71]:

mnb_classifier = MultinomialNB()
mnb_classifier.fit(X_train, y_train)
y_pred = mnb_classifier.predict(X_val)
print(accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))



0.8625
              precision    recall  f1-score   support

           0       0.93      0.87      0.90       550
           1       0.94      0.86      0.90       704
           2       0.71      0.87      0.78       178
           3       0.83      0.87      0.85       275
           4       0.80      0.83      0.81       212
           5       0.62      0.84      0.71        81

    accuracy                           0.86      2000
   macro avg       0.80      0.86      0.83      2000
weighted avg       0.87      0.86      0.87      2000

[[479  10  13  20  19   9]
 [ 11 608  42  20   9  14]
 [  4  15 155   1   1   2]
 [  9   6   5 240  12   3]
 [  9   4   2   8 175  14]
 [  3   5   0   1   4  68]]


# 4. Model Evaluation

In [72]:
# evaluate with test set
y_pred = logreg_classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


0.894
              precision    recall  f1-score   support

           0       0.94      0.93      0.93       581
           1       0.93      0.90      0.91       695
           2       0.72      0.86      0.79       159
           3       0.89      0.89      0.89       275
           4       0.90      0.84      0.87       224
           5       0.69      0.83      0.75        66

    accuracy                           0.89      2000
   macro avg       0.84      0.88      0.86      2000
weighted avg       0.90      0.89      0.90      2000

[[539  15   2  17   7   1]
 [  6 624  49   4   2  10]
 [  3  16 137   3   0   0]
 [ 11  11   2 245   6   0]
 [ 12   3   0   7 188  14]
 [  2   4   0   0   5  55]]


In [73]:
y_pred = mnb_classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


0.858
              precision    recall  f1-score   support

           0       0.93      0.89      0.91       581
           1       0.93      0.85      0.89       695
           2       0.62      0.86      0.72       159
           3       0.85      0.86      0.86       275
           4       0.86      0.81      0.83       224
           5       0.54      0.88      0.67        66

    accuracy                           0.86      2000
   macro avg       0.79      0.86      0.81      2000
weighted avg       0.87      0.86      0.86      2000

[[515  12  10  23  12   9]
 [ 10 589  63   8   8  17]
 [  5  12 136   2   1   3]
 [ 12  11   6 237   8   1]
 [ 11   4   2   7 181  19]
 [  1   4   1   1   1  58]]
